### 2020707035 박시언 Lab4

1. nan null data handling
2. feature to feature corr check and drop
3. feature to Y corr check and select
4. hyperparameter tuning with Bayesian Optimization

In [64]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
encoder = LabelEncoder()
model = xgb.XGBRegressor()
model_param = dict(min_samples_rate=0.9, n_extimators=10, max_depth=6)

data = pd.read_csv('./train.csv')
dum = np.random.rand(len(data), 5)
train_df, test_df , _, __ = train_test_split(data,dum)
object_columns = data.select_dtypes(include=['object']).columns


In [65]:
test_df['LotFrontage']

624      80.0
421       NaN
297      66.0
580       NaN
371      80.0
        ...  
1110      NaN
471      92.0
1232     70.0
55      100.0
1057      NaN
Name: LotFrontage, Length: 365, dtype: float64

In [66]:
train_df.isna().sum()

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      185
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64

In [67]:
test_df.isna().sum()

Id                0
MSSubClass        0
MSZoning          0
LotFrontage      74
LotArea           0
                 ..
MoSold            0
YrSold            0
SaleType          0
SaleCondition     0
SalePrice         0
Length: 81, dtype: int64

In [68]:
train_df.isnull().sum()

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      185
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64

In [69]:
len(train_df)

1095

In [70]:
def NullDropHandler(train, test):
    tmp_stack = []
    
    # 1. train과 test 데이터프레임의 공통 열만 처리
    common_columns = train.columns.intersection(test.columns)

    # 2. 결측값이 200개 이상인 열은 삭제
    for column in common_columns:
        if train[column].isnull().sum() > 200:
            train = train.drop([column], axis=1)
            test = test.drop([column], axis=1)
            tmp_stack.append(column)  # 제거한 열을 스택에 저장

    # 공통 열 리스트를 업데이트하여 삭제된 열을 제외
    common_columns = train.columns.intersection(test.columns)

    # 3. 결측값이 있는 열을 처리
    for column in common_columns:
        # train 데이터프레임에서 결측값이 있는 경우
        if train[column].isnull().sum() > 0:
            if train[column].dtype != 'object':  # 숫자형 데이터 확인
                value_train = train[column].mean()
                train[column] = train[column].fillna(value_train)  
            else:
                train[column] = train[column].fillna(str('none'))  

        # test 데이터프레임에서 결측값이 있는 경우
        if test[column].isnull().sum() > 0:
            if test[column].dtype != 'object':  # 숫자형 데이터 확인
                value_test = test[column].mean()
                test[column] = test[column].fillna(value_test)  
            else:
                test[column] = test[column].fillna(str('none')) 

    return train, test, tmp_stack  # 제거된 열 목록도 반환


In [71]:
len(train_df.columns)

81

In [72]:
print(train_df.isnull().sum().sum())

5866


In [73]:
def NaNHandler(train, test):
    tmp_stack = []
    common_columns = train.columns.intersection(test.columns)   
    # 1. NaN 값이 200개 이상인 열은 삭제
    for column in common_columns:
        if train[column].isna().sum() > 200:
            train = train.drop([column], axis=1)
            test = test.drop([column], axis=1)
            tmp_stack.append(column) 
    common_columns = train.columns.intersection(test.columns)
    # 2. NaN 값이 있는 열을 처리
    for column in common_columns:
        # train 데이터프레임에서 결측값이 있는 경우
        if train[column].isna().sum() > 0:
            if train[column].dtype != 'object':  # 숫자형 데이터 확인
                value_train = train[column].mean()
                train[column] = train[column].fillna(value_train)  # 평균값으로 NaN 채움
            else:
                train[column] = train[column].fillna(str('none')) 

        # test 데이터프레임에서 결측값이 있는 경우
        if test[column].isna().sum() > 0:
            if test[column].dtype != 'object':  # 숫자형 데이터 확인
                value_test = test[column].mean()
                test[column] = test[column].fillna(value_test)  # 평균값으로 NaN 채움
            else:
                test[column] = test[column].fillna(str('none')) 

    return train, test, tmp_stack 


train_df, test_df, removed_columns = NaNHandler(train_df, test_df)


In [74]:
test_df.isna().sum().sum()

np.int64(0)

In [75]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

label_encoder = LabelEncoder()

def Encoder(df1, df2):
    for column in df1.columns:
        if df1[column].dtype == 'object':  # 문자열 타입 확인
            df1[column] = label_encoder.fit_transform(df1[column])
            
            # test_df에 없는 값은 -1로 대체
            df2[column] = df2[column].apply(lambda x: x if x in label_encoder.classes_ else 'none')
            label_encoder.classes_ = np.append(label_encoder.classes_, 'none')
            df2[column] = label_encoder.transform(df2[column])
            
    return df1, df2

train_df, test_df = Encoder(train_df, test_df)


In [76]:
print(train_df.isna().sum().sum())
print(test_df.isna().sum().sum())

0
0


In [77]:
def remove_collinear_features(train_df: pd.DataFrame, test_df: pd.DataFrame, threshold=0.8) -> tuple:
    corr_matrix = train_df.corr()
    iters = range(len(corr_matrix.columns) - 1)
    drop_cols = []

    for i in iters:
        for j in range(i + 1):
            item = corr_matrix.iloc[j:(j + 1), (i + 1):(i + 2)]
            col = item.columns
            row = item.index
            # 절대값을 씌우는 이유는
            # corr 절대값이 높은거를 제거하면 되기 때문에
            val = abs(item.values)

            if val >= threshold:
                print(col.values[0], '|', row.values[0], '|', round(val[0][0], 2))
                drop_cols.append(col.values[0])

    drops = set(drop_cols)
    drops.discard('SalePrice') 
    train_df = train_df.drop(columns=drops)
    test_df = test_df.drop(columns=drops)

    return train_df, test_df
train_df, test_df = remove_collinear_features(train_df,test_df)


Exterior2nd | Exterior1st | 0.87
1stFlrSF | TotalBsmtSF | 0.84
TotRmsAbvGrd | GrLivArea | 0.83
GarageArea | GarageCars | 0.88


In [78]:
Y_train = train_df['SalePrice']
X_train = train_df.drop(['SalePrice'],axis = 1)
Y_test = test_df['SalePrice']
X_test = test_df.drop(['SalePrice'],axis = 1)

In [79]:
numeric_features = X_train.select_dtypes(include=[np.number])

# X와 y 사이의 상관계수 계산
correlation_matrix = numeric_features.corrwith(Y_train)

# 상관계수가 높은 상위 5개의 특성 추출
top_5_features = correlation_matrix.abs().sort_values(ascending=False).head(5)
top_5_features

OverallQual    0.785660
GrLivArea      0.698884
GarageCars     0.634704
BsmtQual       0.610871
ExterQual      0.608780
dtype: float64

In [80]:
set_col = X_train.columns.intersection(object_columns)   
tmp = X_train[set_col]
correlation_matrix = tmp.corrwith(Y_train)
top_5_features_ob = correlation_matrix.abs().sort_values(ascending=False).head(5)
top_5_features_ob

BsmtQual        0.610871
ExterQual       0.608780
KitchenQual     0.591580
GarageFinish    0.543607
GarageType      0.403782
dtype: float64

In [81]:
union_selected_cols = top_5_features.index.union(top_5_features_ob.index)

In [82]:
X_train = X_train[union_selected_cols]
X_test = X_test[union_selected_cols]

In [83]:
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error
import xgboost as xgb
def xgb_evaluate(max_depth, learning_rate, n_estimators, gamma, min_child_weight, subsample, colsample_bytree):
    params = {
        'max_depth': int(max_depth),
        'learning_rate': learning_rate,
        'n_estimators': int(n_estimators),
        'gamma': gamma,
        'min_child_weight': min_child_weight,
        'subsample': subsample,
        'colsample_bytree': colsample_bytree,
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'seed': 42
    }
    
    model = xgb.XGBRegressor(**params)
    model.fit(X_train, Y_train)
    predictions = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(Y_test, predictions))
    return -rmse  # 베이지안 최적화는 최대화를 목표로 하므로 음수로 변환

# 베이지안 최적화 범위 설정
pbounds = {
    'max_depth': (3, 10),
    'learning_rate': (0.01, 0.3),
    'n_estimators': (50, 500),
    'gamma': (0, 5),
    'min_child_weight': (1, 10),
    'subsample': (0.5, 1),
    'colsample_bytree': (0.5, 1)
}

optimizer = BayesianOptimization(f=xgb_evaluate, pbounds=pbounds, random_state=42)
optimizer.maximize(init_points=5, n_iter=25)

# 최적의 하이퍼파라미터 출력
print(optimizer.max)

|   iter    |  target   | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------------------
| 1         | -3.895e+0 | 0.6873    | 4.754     | 0.2223    | 7.191     | 2.404     | 120.2     | 0.529     |
| 2         | -3.708e+0 | 0.9331    | 3.006     | 0.2153    | 3.144     | 9.729     | 424.6     | 0.6062    |
| 3         | -3.587e+0 | 0.5909    | 0.917     | 0.09823   | 6.673     | 4.888     | 181.1     | 0.8059    |
| 4         | -3.413e+0 | 0.5697    | 1.461     | 0.1162    | 6.192     | 8.067     | 139.9     | 0.7571    |
| 5         | -3.918e+0 | 0.7962    | 0.2323    | 0.1862    | 4.194     | 1.585     | 477.0     | 0.9828    |
| 6         | -3.431e+0 | 0.6909    | 2.055     | 0.1084    | 6.844     | 8.534     | 139.8     | 0.9432    |
| 7         | -3.329e+0 | 0.5       | 0.0       | 0.1512    | 3.0       | 5.304     | 149.9     | 0.5       |
| 8       

In [84]:
best_params = optimizer.max['params']
best_params['max_depth'] = int(best_params['max_depth'])  # max_depth는 정수형이어야 함
best_params['n_estimators'] = int(best_params['n_estimators'])  # n_estimators도 정수형이어야 함

final_model = xgb.XGBRegressor(
    max_depth=best_params['max_depth'],
    learning_rate=best_params['learning_rate'],
    n_estimators=best_params['n_estimators'],
    gamma=best_params['gamma'],
    min_child_weight=best_params['min_child_weight'],
    subsample=best_params['subsample'],
    colsample_bytree=best_params['colsample_bytree'],
    objective='reg:squarederror',
    seed=42
)

final_model.fit(X_train, Y_train)
final_predictions = final_model.predict(X_test)
final_rmse = np.sqrt(mean_squared_error(Y_test, final_predictions))

print(f"최종 모델의 RMSE: {final_rmse}")

최종 모델의 RMSE: 31854.99106116922
